In [2]:
#Import external functions
from image_processing import *
from video_processing import *
from utils import *
from custom_models import *
import pickle
import random
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint

Using TensorFlow backend.


In [3]:
# Load images for training
data_type = 'flow_images'
weight_file_path = 'saved_models/caffenet_single_flow.hdf5'

classes_dictionary = pickle.load(open('data_a/action_dictionary.p','rb'))

In [4]:
# Single frame model definition
input_shape = (crop_height,crop_width,3)
seq_input = (seq_len,crop_height,crop_width,3)

#SGD Optimizer Parameters
base_lr = 0.001
momentum = 0.9

#Step-based LearningRate Parameters
epochs = 420
gamma = 0.1
stepsize = 210

In [5]:
#Create CNN-LSTM model
lstm_model = LSTMCaffeDonahueFunctional(seq_input=seq_input, num_labels=num_labels)

#Load Single Frame weights for faster training
lstm_model = load_single_frame_weights(lstm_model, weight_file_path)

# Configure the model for training
lstm_model = model_compile(model=lstm_model, base_lr=base_lr, momentum=momentum)

In [6]:
file = open('data_a/ucf101_split1_trainVideos.txt','r')
train_videos = file.readlines()
file.close()
N = len(train_videos)
random.shuffle(train_videos)
val_videos = train_videos[:N//3]
train_videos = train_videos[N//3:]

batch_size_train = 17
batch_size_val = 17

In [7]:
#### STEP-BASED DECAY
# To Keras ston SGD apo mono tou kanei time-based decay
# Protimoume Step decay, giati emfanizei kalutero convergence
# Isws 8a mporousa na dokimasw kai Adaptive Learning
#
# Possible solution : https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# Possible solution : https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
####
def step_decay(epoch):
    initial_lrate = base_lr
    drop = gamma
    epochs_drop = stepsize
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

In [ ]:
for pseudoseed in range(iterations):
    train_generator = frame_generator(train_videos,batch_size_train,data_type,pseudoseed)
    val_generator = frame_generator(val_videos,batch_size_val,data_type,pseudoseed)

    filepath = 'saved_models/lstm_flow_512_pseudo'+str(pseudoseed)+'.hdf5'
    checkpointer = ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True)
    loss_history = LossHistory()
    lrate = LearningRateScheduler(step_decay,verbose=1)

    history = lstm_model.fit_generator(generator=train_generator,
                                       steps_per_epoch=15,
                                       validation_data=val_generator,
                                       validation_steps=5,
                                       epochs=epochs,
                                       callbacks=[lrate,loss_history,checkpointer])

Epoch 1/420

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 35s 2s/step - loss: 5.1960 - acc: 0.0235 - top_k_categorical_accuracy: 0.0863 - val_loss: 5.1963 - val_acc: 0.0118 - val_top_k_categorical_accuracy: 0.0941

Epoch 00001: val_loss improved from inf to 5.19627, saving model to saved_models/lstm_flow_512_pseudo0.hdf5
Epoch 2/420

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 5.1883 - acc: 0.0275 - top_k_categorical_accuracy: 0.1373 - val_loss: 5.1813 - val_acc: 0.0824 - val_top_k_categorical_accuracy: 0.2000

Epoch 00002: val_loss improved from 5.19627 to 5.18129, saving model to saved_models/lstm_flow_512_pseudo0.hdf5
Epoch 3/420

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 5.1631 - acc: 0.0941 - top_k_categorical_accuracy: 0.2510 - val_loss: 5.1516 - val_

15/15 [==============================] - 24s 2s/step - loss: 1.8193 - acc: 0.6863 - top_k_categorical_accuracy: 0.9216 - val_loss: 1.9037 - val_acc: 0.6118 - val_top_k_categorical_accuracy: 0.8941

Epoch 00045: val_loss improved from 1.96452 to 1.90366, saving model to saved_models/lstm_flow_512_pseudo0.hdf5
Epoch 46/420

Epoch 00046: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 2.0469 - acc: 0.6471 - top_k_categorical_accuracy: 0.8706 - val_loss: 2.3839 - val_acc: 0.5765 - val_top_k_categorical_accuracy: 0.8235

Epoch 00046: val_loss did not improve from 1.90366
Epoch 47/420

Epoch 00047: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.8314 - acc: 0.6980 - top_k_categorical_accuracy: 0.9059 - val_loss: 2.2306 - val_acc: 0.6235 - val_top_k_categorical_accuracy: 0.8235

Epoch 00047: val_loss did not improve from 1.90366
Epoch 48/420

Epoch 00048: Lea


Epoch 00068: val_loss did not improve from 1.53067
Epoch 69/420

Epoch 00069: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.6593 - acc: 0.6941 - top_k_categorical_accuracy: 0.9333 - val_loss: 1.5861 - val_acc: 0.7176 - val_top_k_categorical_accuracy: 0.9176

Epoch 00069: val_loss did not improve from 1.53067
Epoch 70/420

Epoch 00070: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.5574 - acc: 0.7647 - top_k_categorical_accuracy: 0.9373 - val_loss: 1.9312 - val_acc: 0.6353 - val_top_k_categorical_accuracy: 0.9412

Epoch 00070: val_loss did not improve from 1.53067
Epoch 71/420

Epoch 00071: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.6133 - acc: 0.7529 - top_k_categorical_accuracy: 0.9216 - val_loss: 2.0449 - val_acc: 0.6588 - val_top_k_categorical_accuracy: 0.8706

Epoch 000


Epoch 00117: val_loss did not improve from 1.35035
Epoch 118/420

Epoch 00118: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.2643 - acc: 0.8510 - top_k_categorical_accuracy: 0.9686 - val_loss: 1.6157 - val_acc: 0.7294 - val_top_k_categorical_accuracy: 0.9294

Epoch 00118: val_loss did not improve from 1.35035
Epoch 119/420

Epoch 00119: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.2582 - acc: 0.8510 - top_k_categorical_accuracy: 0.9765 - val_loss: 1.6683 - val_acc: 0.7882 - val_top_k_categorical_accuracy: 0.9176

Epoch 00119: val_loss did not improve from 1.35035
Epoch 120/420

Epoch 00120: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.1995 - acc: 0.8392 - top_k_categorical_accuracy: 0.9882 - val_loss: 1.7662 - val_acc: 0.7176 - val_top_k_categorical_accuracy: 0.9176

Epoch 

15/15 [==============================] - 24s 2s/step - loss: 1.1069 - acc: 0.8706 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.4293 - val_acc: 0.8000 - val_top_k_categorical_accuracy: 0.9529

Epoch 00167: val_loss did not improve from 1.18268
Epoch 168/420

Epoch 00168: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.1298 - acc: 0.8706 - top_k_categorical_accuracy: 0.9843 - val_loss: 1.3637 - val_acc: 0.8588 - val_top_k_categorical_accuracy: 0.9412

Epoch 00168: val_loss did not improve from 1.18268
Epoch 169/420

Epoch 00169: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.1836 - acc: 0.8667 - top_k_categorical_accuracy: 0.9804 - val_loss: 1.5681 - val_acc: 0.8235 - val_top_k_categorical_accuracy: 0.9176

Epoch 00169: val_loss did not improve from 1.18268
Epoch 170/420

Epoch 00170: LearningRateScheduler setting learning rate to 0.001.
15/15 [

...

15/15 [==============================] - 24s 2s/step - loss: 0.7698 - acc: 0.9569 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.4559 - val_acc: 0.7412 - val_top_k_categorical_accuracy: 0.9412

Epoch 00051: val_loss did not improve from 1.12798
Epoch 52/420

Epoch 00052: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.8058 - acc: 0.9373 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.3402 - val_acc: 0.8353 - val_top_k_categorical_accuracy: 0.9294

Epoch 00052: val_loss did not improve from 1.12798
Epoch 53/420

Epoch 00053: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7729 - acc: 0.9529 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.1343 - val_acc: 0.8353 - val_top_k_categorical_accuracy: 0.9882

Epoch 00053: val_loss did not improve from 1.12798
Epoch 54/420

Epoch 00054: LearningRateScheduler setting learning rate to 0.001.
15/15 [===

15/15 [==============================] - 24s 2s/step - loss: 0.7448 - acc: 0.9569 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.2179 - val_acc: 0.8706 - val_top_k_categorical_accuracy: 0.9412

Epoch 00076: val_loss did not improve from 1.09122
Epoch 77/420

Epoch 00077: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7959 - acc: 0.9333 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.6307 - val_acc: 0.7294 - val_top_k_categorical_accuracy: 0.9294

Epoch 00077: val_loss did not improve from 1.09122
Epoch 78/420

Epoch 00078: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7905 - acc: 0.9373 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.5093 - val_acc: 0.7176 - val_top_k_categorical_accuracy: 0.9412

Epoch 00078: val_loss did not improve from 1.09122
Epoch 79/420

Epoch 00079: LearningRateScheduler setting learning rate to 0.001.
15/15 [===


Epoch 00100: val_loss did not improve from 1.04680
Epoch 101/420

Epoch 00101: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7607 - acc: 0.9569 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.6369 - val_acc: 0.7882 - val_top_k_categorical_accuracy: 0.9059

Epoch 00101: val_loss did not improve from 1.04680
Epoch 102/420

Epoch 00102: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7157 - acc: 0.9686 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.4267 - val_acc: 0.7765 - val_top_k_categorical_accuracy: 0.9176

Epoch 00102: val_loss did not improve from 1.04680
Epoch 103/420

Epoch 00103: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.6989 - acc: 0.9765 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.2246 - val_acc: 0.8706 - val_top_k_categorical_accuracy: 0.9647

Epoch 

15/15 [==============================] - 25s 2s/step - loss: 0.7234 - acc: 0.9725 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.4668 - val_acc: 0.7647 - val_top_k_categorical_accuracy: 0.9529

Epoch 00125: val_loss did not improve from 0.97566
Epoch 126/420

Epoch 00126: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 25s 2s/step - loss: 0.7381 - acc: 0.9647 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.3410 - val_acc: 0.8235 - val_top_k_categorical_accuracy: 0.9529

Epoch 00126: val_loss did not improve from 0.97566
Epoch 127/420

Epoch 00127: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.7016 - acc: 0.9686 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.2702 - val_acc: 0.7882 - val_top_k_categorical_accuracy: 0.9765

Epoch 00127: val_loss did not improve from 0.97566
Epoch 128/420

Epoch 00128: LearningRateScheduler setting learning rate to 0.001.
15/15 [

15/15 [==============================] - 25s 2s/step - loss: 0.7151 - acc: 0.9647 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.2657 - val_acc: 0.8000 - val_top_k_categorical_accuracy: 0.9529

Epoch 00150: val_loss did not improve from 0.97566
Epoch 151/420

Epoch 00151: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 25s 2s/step - loss: 0.7055 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.2858 - val_acc: 0.8000 - val_top_k_categorical_accuracy: 0.9529

Epoch 00151: val_loss did not improve from 0.97566
Epoch 152/420

Epoch 00152: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 25s 2s/step - loss: 0.6894 - acc: 0.9804 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.3748 - val_acc: 0.8235 - val_top_k_categorical_accuracy: 0.9294

Epoch 00152: val_loss did not improve from 0.97566
Epoch 153/420

Epoch 00153: LearningRateScheduler setting learning rate to 0.001.
15/15 [